<a href="https://colab.research.google.com/github/kirillel85/kirillel85/blob/public_main/x5_task_diag_sankey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The full text of [Step Sankey](https://doc.retentioneering.com/stable/doc/user_guides/step_sankey.html) user guide is available on the retentioneering website.

Задача:
Для обеспечения потребности на распределительный центр на 49 неделю (номер недели ISO) заказали различные товары. По определенным причинам весь запланированный товар не был доставлен на плановой неделе. В таблице на листе "таблица тест 3" данные по заказам, с которыми возникли проблемы. На любой BI-платформе (PowerBI Desktop, Tableu, Yandex DataLens или другая) постройте диаграмму Sankey с потоками объемов товаров не поставленных на 49 плановой неделе на распределительный центр. При необходимости провести дополнительные вычисления допустимо в Excel. Измерениями Sankey-диаграммы должны стать столбцы "Проблема", "Группа проблем" и "Ответственный департамент". При выполнении стоит учесть, что диаграмма может быть продемонстрирована на собрании директору организации. Результат приложите в виде файла, экспортированного из BI-системы.

In [ ]:
!pip install retentioneering
# Установка библиотеки retentioneering

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import retentioneering
from retentioneering import datasets
import matplotlib.pyplot as plt
from retentioneering.eventstream import Eventstream

In [ ]:
! gdown --id 1WrYjFIX62_q62O-vB76gWOVq696F6CIX
# Загружаем эксель файл

In [5]:
proba = pd.read_excel('/content/Тестовые задания.xlsx', sheet_name='таблица тест 3')
# Читаем

In [6]:
proba.columns = ['orders_id', 'date_plan',
       'date_fact', 'good_id', 'amount_ordered',
       'amount_accepted', 'problem', 'group_problem',
       'depatment']
# Переименовываем

In [7]:
proba['underloading'] = proba['amount_ordered'] - proba['amount_accepted']
# Создаём расчетное поле недогруз

In [8]:
proba.date_fact = pd.to_datetime(proba['date_fact'], format='%Y-%M-%D')
# Переводим в формат даты

In [9]:
proba.insert(3, "date_fact_week", proba['date_fact'].dt.week)
# Вставляем поле с номером недели в датасет

<ipython-input-9-8dde8d61e871>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  proba.insert(3, "date_fact_week", proba['date_fact'].dt.week)


In [10]:
proba = proba[proba['date_fact_week'] == 49].sort_values(by='problem')
# Сортируем по 49 неделе

In [11]:
proba.problem = proba.problem.str.replace('Проблемы с качеством товара', 'Качество')
# Готов

In [12]:
proba = proba[['orders_id','date_fact','problem', 'group_problem', 'depatment','underloading']]
part1 = proba[['orders_id','date_fact','problem']]
part1.columns = ['orders_id', 'date_fact', 'event_name']
part2 = proba[['orders_id','date_fact','group_problem']]
part2.columns = ['orders_id', 'date_fact', 'event_name']
part3 = proba[['orders_id','date_fact','depatment']]
part3.columns = ['orders_id', 'date_fact', 'event_name']
proba4 = pd.concat([part1,part2,part3])

In [13]:
# Создаём схему ключей инвентов для потока
raw_data_schema = {
    'user_id': 'orders_id',
    'event_name': 'event_name',
    'event_timestamp': 'date_fact'
    }

# Загружаем в конструктор потока.
stream_proba = Eventstream(proba4, raw_data_schema=raw_data_schema)


In [14]:
stream_proba.step_sankey(max_steps=5);
# @title Создаём диаграмму Санкей

In [15]:
# stream_proba.to_dataframe()

In [16]:
# stream_proba\
#     .step_sankey(show_plot=False)\
#     .values[0]